In [1]:
import yaml
import re
import os

#Set to your local directory 
yaml_file = 'C:/Users/jocke/Documents/GitHub/Misc/Brewery/DrinkConfig.yaml'
destination = 'C:/Users/jocke/Documents/GitHub/Misc/Brewery/FlatFiles/'

In [2]:
class Recipe:
    def __init__(self, name: str, cookingtime: int, ingredients: list, difficulty: int, **kwargs):
        # required params
        self.name = name
        self.cookingtime = cookingtime
        self.ingredients = ingredients
        self.difficulty = difficulty

        # optional params
        self.distilltime = kwargs.get('distilltime', None)
        self.distillruns = kwargs.get('distillruns', None)
        self.wood = kwargs.get('wood', None)
        self.age = kwargs.get('age', None)
        self.alcohol = kwargs.get('alcohol', None)
        self.effects = kwargs.get('effects', None)

    def difficulty_word(self):
        difficulty_levels = ["rather simple","relatively straightforward","easy","of moderate difficulty"
            ,"of intermediate difficulty","somewhat challenging","rather difficult","very complex","quite hard","one of the most difficult"]
        return difficulty_levels[self.difficulty-1]
    
    def wood_word(self):
        wood_types = [" birch","n oak"," jungle"," spruce","n acacia"," dark oak"," crimson"," warped"," mangrove"," cherry"," bamboo"," cut copper"]
        return wood_types[self.wood]

In [6]:
#Read in YML config file, store in data, if there is an error calling this function, check your yaml file for un-indented items
def parse_yaml():
    recipes:list[Recipe] = []

    with open(yaml_file, 'r') as file:
        data = yaml.safe_load(file)

    for recipe in data:
        current_recipe = Recipe(
            name=recipe,
            cookingtime=int(data[recipe]["cookingtime"]),
            ingredients=data[recipe]["ingredients"],
            difficulty=int(data[recipe]["difficulty"]),
        )

        for key in ['distilltime', 'distillruns', 'wood', 'age', 'alcohol', 'effects']:
            if key in data[recipe]:
                setattr(current_recipe, key, data[recipe][key])

        recipes.append(current_recipe)
    
    return recipes

In [4]:
#Generates book given a particlar recipe
def generate_book(recipe:Recipe):
    #required params
    content="<AutoPage><AutoAlias><ReqPermission>\n"
    seperated_name=re.sub(r'([a-z])([A-Z])', r'\1 \2', recipe.name)
    content+=f"&l{seperated_name}\n"
    content+=f"Ingredients:\n"
    for ingredient in recipe.ingredients:
        content+=f"&o- {ingredient.split('/')[1]} {ingredient.split('/')[0]}\n"
    content+=f"<NextPage>\n"
    content+=f"This recipe will require &l{recipe.cookingtime} &0minutes, and is &l{recipe.difficulty_word()}&0 to create.\n"
    #optional params
    if recipe.wood !=None:
        if recipe.wood == 0:
            content+=f"One can use any type of barrel, but must age for {recipe.age} days."
        else:
            content+=f"It is best to use a{recipe.wood_word()} wood barrel, and age for {recipe.age} days.\n"
    if recipe.distillruns!=None:
        content+=f"Master brewers have found that it must be distilled {recipe.distillruns} times for {recipe.distilltime} seconds or so..."
    if recipe.alcohol!=None:
        content+=f"Properly brewed, this will have an alcohol density of {recipe.alcohol}\n"
    if recipe.effects!=None:
        content+="\nEffects include:\n"
        for effects in recipe.effects:
            content+=f"&o- {effects.split('/')[0].lower()}\n"
    if recipe.alcohol!=None:
        content+="<NextPage>\n&9Tip: Eating bread or drinking milk may sober you at a slightly increased rate!\n&1This is a virtual alcoholic beverage recipe!We do not advocate for underage drinking in real life. Please drink responsibly."

    file_path = f"{destination}{recipe.name}.txt"
    with open(file_path,'w',newline="\n") as file:
        file.write(content)

In [5]:
#Call function to convert yaml into Recipe objects
recipes:list[Recipe] = parse_yaml()
#Iterate through Recipes, creating a book for each
for recipe in recipes:
    try:
        generate_book(recipe)
    except Exception as e:
        print(f"There was an issue creating {recipe.name}! Please take a look.")
        print(e)